# Introduction to Transformations
This example introduces the `pybop.BaseTransformation` class and its instances. This class enables the cost and likelihood functions to be evaluated in a transformed, search space. The search space is used by the optimiser and sampler classes during inference. These transformations can be linear (e.g. `pybop.ScaledTransformation`) or non-linear (e.g. `pybop.LogTransformation`). By default, if transformations are applied, the sampling and optimisers will search in the transformed space.

Transformations can be helpful when the difference in parameter magnitudes is large, or to create a search space that is better posed for the optimisation algorithm.

### Setting up the Environment

If you don't already have PyBOP installed, check out the [installation guide](https://pybop-docs.readthedocs.io/en/latest/installation.html) first.

We begin by importing the necessary libraries. Let's also fix the random seed to generate consistent output during development.

In [ ]:
import numpy as np
import pybamm

import pybop

pybop.plot.PlotlyManager().pio.renderers.default = "notebook_connected"

np.random.seed(8)  # users can remove this line

First, to showcase the transformation functionality, we need to construct an optimisation problem. This class is typically built on the following objects:
- Model
- Dataset
- Parameters to identify
- Cost function

We will first construct the model, then the parameters and corresponding dataset. Once that is complete, the problem will be created with which we will showcase the different interactions users can have with transformations.

In [ ]:
model = pybamm.lithium_ion.SPM()

Now that we have the model constructed, let's define the parameters for identification. At this point, we define the transformations applied to each parameter. PyBOP allows for transformations to be applied at the individual parameter level, which are then combined for application during the optimisation.

## Linear transformation

Below we apply a linear transformation using the `pybop.ScaledTransformation` class. This class has arguments for a `coefficient` which defines the linear stretch or scaling of the search space, and `intercept` which defines the translation or shift. The equation for this transformation is:

$$
y_{search} = m(x_{model}+b)
$$

where $m$ is the linear scale coefficient, $b$ is the intercept, $x_{model}$ is the model parameter space, and $y_{search}$ is the transformed space.

In [ ]:
parameters = [
    pybop.Parameter(
        "Negative electrode active material volume fraction",
        initial_value=0.6,
        bounds=[0.35, 0.7],
        transformation=pybop.ScaledTransformation(
            coefficient=1 / 0.35, intercept=-0.35, n_parameters=1
        ),
    ),
    pybop.Parameter(
        "Positive electrode active material volume fraction",
        initial_value=0.6,
        bounds=[0.45, 0.625],
        transformation=pybop.ScaledTransformation(
            coefficient=1 / 0.175, intercept=-0.45, n_parameters=1
        ),
    ),
]

Next, to create the `pybop.Dataset` we generate some synthetic data from the model using a `pybamm.Simulation`.

In [ ]:
t_eval = np.linspace(0, 10, 100)
sim = pybamm.Simulation(model)
sol = sim.solve(t_eval=t_eval)

dataset = pybop.Dataset(
    {
        "Time [s]": t_eval,
        "Current function [A]": sol["Current [A]"](t_eval),
        "Voltage [V]": sol["Voltage [V]"](t_eval),
    }
)

/home/nicola/GitHub/PyBOP/.nox/notebooks-overwrite/lib/python3.12/site-packages/pybamm/simulation.py:122: UserWarning:

The default solver changed to IDAKLUSolver after the v25.4.0. release. You can swap back to the previous default by using `pybamm.CasadiSolver()` instead.



Now that we have the model, parameters, and dataset, we can combine them and construct the problem class with a cost function.

In [ ]:
builder = (
    pybop.builders.Pybamm()
    .set_dataset(dataset)
    .set_simulation(model)
    .add_cost(pybop.costs.pybamm.SumOfPower("Voltage [V]"))
)
for param in parameters:
    builder.add_parameter(param)

problem = builder.build()

To evaluate the cost, we pass parameter values in the original (untransformed) model space.

In [ ]:
problem.set_params([0.6, 0.6])
cost_value = problem.run()
print("Cost:", cost_value)

Cost: [0.00690259]


Now we can set up and run the optimisation process, which will apply the transformations internally.

In [ ]:
options = pybop.ScipyMinimizeOptions(maxiter=50)
optim = pybop.SciPyMinimize(problem, options=options)
results = optim.run()

To see the effect of transformation, we can compare the cost landscapes plotted in the model and search spaces. Let's first plot the cost in the model space through the conventional method:

In [ ]:
pybop.plot.contour(optim);

Next, we can use the `apply_transform` argument when constructing the cost landscape to plot in the transformed space.

In [ ]:
pybop.plot.contour(optim, apply_transform=True);

Note the difference in axis scale compared to the non-transformed landscape

## Log transformation

Next, let's change the transformation on the "Positive electrode active material volume fraction" to a non-linear, log transformation.

In [ ]:
old_param = parameters[1]
parameters[1] = pybop.Parameter(
    "Positive electrode active material volume fraction",
    initial_value=old_param.initial_value,
    bounds=old_param.bounds,
    transformation=pybop.LogTransformation(),
)

builder = (
    pybop.builders.Pybamm()
    .set_dataset(dataset)
    .set_simulation(model)
    .add_cost(pybop.costs.pybamm.SumOfPower("Voltage [V]"))
)
for param in parameters:
    builder.add_parameter(param)

problem = builder.build()

Let's plot the cost landscape again. This time, the values on the y-axis are negative as they correspond to the log of the model values.

In [ ]:
pybop.plot.contour(problem, apply_transform=True);

## Concluding thoughts

In this notebook, we've introduced the transformation class and its interaction with the parameters. Transformations allow the optimisation or sampling search space to be transformed for improved convergence in situations where the optimisation hyperparameters are poorly tuned, or in optimisation tasks with high variance in the parameter magnitudes. 